# How many mosquitoes would it take to exsanguinate someone?

We're going to approach this mathmatically using a number of assumptions.

First, let's import pandas.

In [1]:
import pandas as pd

Now, our assumptions. These are all based on only very basic research online and all volumes are in millilitres.

In [4]:
human_blood_capacity = 5000

human_blood_regen = 1000
human_blood_regen_upper = 2000
human_blood_regen_lower = 400

blood_loss_maximum = human_blood_capacity * .4

In [5]:
df = pd.DataFrame()

df["mosquitoes"] = range(1, 1000000)
df["blood_drain"] = df['mosquitoes'] * -0.005
df["net_blood_regen"] = human_blood_regen + df["blood_drain"]
df["net_blood_regen_lower"] = human_blood_regen_lower + df["blood_drain"]
df["net_blood_regen_upper"] = human_blood_regen_upper + df["blood_drain"]


# for i in range(1, len(df)):
#   previous_cell = df.loc[i-1, 'blood_end_of_day']
#   if  previous_cell >= 0:
#     df.loc[i, 'blood_end_of_day'] = previous_cell + df.loc[i, 'net_daily_blood_regen']
#   else:
#     break

# df["blood_end_of_day"] = df["blood_end_of_day"].clip(upper=human_blood_capacity)
# df = df.dropna()

display(df)
df.to_csv('mosquito.csv')

,mosquitoes,blood_drain,net_blood_regen,net_blood_regen_lower,net_blood_regen_upper
0,1,-0.005,999.995,399.995,1999.995
1,2,-0.010,999.990,399.990,1999.990
2,3,-0.015,999.985,399.985,1999.985
3,4,-0.020,999.980,399.980,1999.980
4,5,-0.025,999.975,399.975,1999.975
...,...,...,...,...,...
999994,999995,-4999.975,-3999.975,-4599.975,-2999.975
999995,999996,-4999.980,-3999.980,-4599.980,-2999.980
999996,999997,-4999.985,-3999.985,-4599.985,-2999.985
999997,999998,-4999.990,-3999.990,-4599.990,-2999.990


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Data/exports/mosquito.csv'

In [9]:
df = pd.read_csv('/content/drive/MyDrive/Data/exports/mosquito.csv')

df = df[(df["net_blood_regen_upper"] >= (-human_blood_capacity-500)) | (df["net_blood_regen_lower"] >= (-human_blood_capacity-500))]
df = df.drop(columns=["Unnamed: 0"])
df = df.iloc[::100, :]

df["human_regen"] = 1000
display(df)
df.to_csv('data.csv')

,mosquitoes,blood_drain,net_blood_regen,net_blood_regen_lower,net_blood_regen_upper,human_regen
0,1,-0.005,999.995,399.995,1999.995,1000
100,101,-0.505,999.495,399.495,1999.495,1000
200,201,-1.005,998.995,398.995,1998.995,1000
300,301,-1.505,998.495,398.495,1998.495,1000
400,401,-2.005,997.995,397.995,1997.995,1000
...,...,...,...,...,...,...
999500,999501,-4997.505,-3997.505,-4597.505,-2997.505,1000
999600,999601,-4998.005,-3998.005,-4598.005,-2998.005,1000
999700,999701,-4998.505,-3998.505,-4598.505,-2998.505,1000
999800,999801,-4999.005,-3999.005,-4599.005,-2999.005,1000
